# Movie Metrics

Data source: Maven Analytics\
Link: [Maven Analytics link](https://maven-datasets.s3.us-east-1.amazonaws.com/Data+Drills/user_activity.zip)

## Your Objective

You've been given a table of Netflix users and another with their viewing activity, including the movie name, date started, and whether they finished it.

Your task is to engineer these new features for each user, based on their activity:
- Date from the first movie they finished
- Name of the first movie they finished
- Date from the last movie they finished
- Name of the last movie they finished
- Movies started
- Movies finished

### Import Libraries

In [48]:
import numpy as np
import pandas as pd

### Loading Datasets

In [49]:
activity_data = pd.read_csv('activity.csv')
users_data = pd.read_csv('users.csv')

activity = activity_data.copy()
users = users_data.copy()

### Explore Datasets

In [50]:
activity.head()

,id,user_id,date,movie_name,finished
0,1,2,2023-06-22,The Shawshank Redemption,1
1,2,1,2023-07-23,Shrek,0
2,3,4,2023-07-27,Fight Club,1
3,4,1,2023-08-23,Top Gun: Maverick,0
4,5,4,2023-08-24,Oppenheimer,0


In [51]:
users.head()

,id,created_at,country_code
0,1,2023-05-26,CA
1,2,2023-06-15,CA
2,3,2023-07-18,MX
3,4,2023-07-27,CA
4,5,2023-09-01,US


In [52]:
# Check for missing values
print("Activity missing values: ", activity.isna().sum().sum())
print("Users missing values: ", users.isna().sum().sum())

Activity missing values:  0
Users missing values:  0


In [53]:
# Converting 'date' column to datetime format
activity['date'] = pd.to_datetime(activity['date'])
users['created_at'] = pd.to_datetime(users['created_at'])

### Features Engineering

#### Option 1: Using different dataset and merge them

In [54]:
# Filter finished movies
finished_movies = activity[activity['finished'] == 1]
    
# Sort by user_id and date to get chronological order
finished_movies = finished_movies.sort_values(['user_id', 'date'])
    
# Get first finished movie per user
first_finished = finished_movies.groupby('user_id').first().reset_index()
first_finished = first_finished[['user_id', 'date', 'movie_name']].rename(columns={
    'date': 'first_date',
    'movie_name': 'first_name'
})
    
# Get last finished movie per user
last_finished = finished_movies.groupby('user_id').last().reset_index()
last_finished = last_finished[['user_id', 'date', 'movie_name']].rename(columns={
    'date': 'last_date', 
    'movie_name': 'last_name'
})

In [55]:
# Count movies started and finished per user
user_counts = activity.groupby('user_id').agg({
    'id': 'count',  # Total movies started
    'finished': 'sum'  # Total movies finished
}).reset_index()
    
user_counts = user_counts.rename(columns={
    'id': 'started',
    'finished': 'finished'
})

In [56]:
# Merge all features together
activity_features = user_counts.merge(first_finished, on='user_id', how='left')
activity_features = activity_features.merge(last_finished, on='user_id', how='left')
    
# Reorder columns for better readability
column_order = [
    'user_id',
    'first_date',
    'first_name', 
    'last_date',
    'last_name',
    'started',
    'finished'
]
    
activity_features = activity_features[column_order]

In [57]:
activity_features

,user_id,first_date,first_name,last_date,last_name,started,finished
0,1,2023-09-12,Turning Red,2025-03-26,Her,30,26
1,2,2023-06-22,The Shawshank Redemption,2025-05-01,Fight Club,15,12
2,3,2023-11-10,Oppenheimer,2025-03-31,Nope,10,7
3,4,2023-07-27,Fight Club,2025-05-09,Avengers: Endgame,43,34
4,5,2023-09-07,Top Gun: Maverick,2025-03-14,Bohemian Rhapsody,31,25
5,6,2023-12-18,Inception,2025-03-27,The Revenant,13,11
6,7,2024-08-28,Avengers: Endgame,2024-09-15,Soul,2,2
7,8,2024-01-24,Arrival,2025-04-25,Whiplash,29,23
8,9,2024-02-25,Big Hero 6,2025-04-11,Moonlight,17,12
9,10,2024-05-01,Whiplash,2025-03-08,Knives Out,7,6


In [58]:
final_merge = pd.merge(
    users, activity_features, 
    on=None, 
    how='left', 
    left_on='id', 
    right_on='user_id', 
    suffixes=('_x', '_y'))
final_merge = final_merge.drop(columns=['country_code', 'user_id'])
final_merge


,id,created_at,first_date,first_name,last_date,last_name,started,finished
0,1,2023-05-26,2023-09-12,Turning Red,2025-03-26,Her,30,26
1,2,2023-06-15,2023-06-22,The Shawshank Redemption,2025-05-01,Fight Club,15,12
2,3,2023-07-18,2023-11-10,Oppenheimer,2025-03-31,Nope,10,7
3,4,2023-07-27,2023-07-27,Fight Club,2025-05-09,Avengers: Endgame,43,34
4,5,2023-09-01,2023-09-07,Top Gun: Maverick,2025-03-14,Bohemian Rhapsody,31,25
5,6,2023-11-20,2023-12-18,Inception,2025-03-27,The Revenant,13,11
6,7,2023-11-21,2024-08-28,Avengers: Endgame,2024-09-15,Soul,2,2
7,8,2024-01-12,2024-01-24,Arrival,2025-04-25,Whiplash,29,23
8,9,2024-01-17,2024-02-25,Big Hero 6,2025-04-11,Moonlight,17,12
9,10,2024-02-13,2024-05-01,Whiplash,2025-03-08,Knives Out,7,6


### Option 2: Using aggregation

In [59]:
# Making a copy of activity dataset
activity2 = activity.copy()

# Coverting Data into pandas date
activity2['date'] = pd.to_datetime(activity2['date'])

# Sorting the dataset
activity2 = activity2.sort_values(['user_id', 'date'])

# Create finished movie mask for conditional aggregation
activity2['finished_date'] = activity2['date'].where(activity2['finished'] == 1)
activity2['finished_movie'] = activity2['movie_name'].where(activity2['finished'] == 1)

activity_features2 = activity2.groupby('user_id').agg({
        'finished_date': ['first', 'last'],
        'finished_movie': ['first', 'last'],
        'id': 'count',
        'finished': 'sum'
    })

# Flatten column names
activity_features2.columns = [
    'first_date', 'last_date',
    'first_movie', 'last_movie', 
    'started', 'finished'
    ]
    
# Reset index to make user_id a column
activity_features2 = activity_features2.reset_index()

activity_features2

,user_id,first_date,last_date,first_movie,last_movie,started,finished
0,1,2023-09-12,2025-03-26,Turning Red,Her,30,26
1,2,2023-06-22,2025-05-01,The Shawshank Redemption,Fight Club,15,12
2,3,2023-11-10,2025-03-31,Oppenheimer,Nope,10,7
3,4,2023-07-27,2025-05-09,Fight Club,Avengers: Endgame,43,34
4,5,2023-09-07,2025-03-14,Top Gun: Maverick,Bohemian Rhapsody,31,25
5,6,2023-12-18,2025-03-27,Inception,The Revenant,13,11
6,7,2024-08-28,2024-09-15,Avengers: Endgame,Soul,2,2
7,8,2024-01-24,2025-04-25,Arrival,Whiplash,29,23
8,9,2024-02-25,2025-04-11,Big Hero 6,Moonlight,17,12
9,10,2024-05-01,2025-03-08,Whiplash,Knives Out,7,6


In [60]:
final_merge2 = pd.merge(
    users, activity_features2, 
    on=None, 
    how='left', 
    left_on='id', 
    right_on='user_id', 
    suffixes=('_x', '_y'))
final_merge2 = final_merge2.drop(columns=['country_code', 'user_id'])
final_merge2

,id,created_at,first_date,last_date,first_movie,last_movie,started,finished
0,1,2023-05-26,2023-09-12,2025-03-26,Turning Red,Her,30,26
1,2,2023-06-15,2023-06-22,2025-05-01,The Shawshank Redemption,Fight Club,15,12
2,3,2023-07-18,2023-11-10,2025-03-31,Oppenheimer,Nope,10,7
3,4,2023-07-27,2023-07-27,2025-05-09,Fight Club,Avengers: Endgame,43,34
4,5,2023-09-01,2023-09-07,2025-03-14,Top Gun: Maverick,Bohemian Rhapsody,31,25
5,6,2023-11-20,2023-12-18,2025-03-27,Inception,The Revenant,13,11
6,7,2023-11-21,2024-08-28,2024-09-15,Avengers: Endgame,Soul,2,2
7,8,2024-01-12,2024-01-24,2025-04-25,Arrival,Whiplash,29,23
8,9,2024-01-17,2024-02-25,2025-04-11,Big Hero 6,Moonlight,17,12
9,10,2024-02-13,2024-05-01,2025-03-08,Whiplash,Knives Out,7,6


## Extra Question

### How many users have "Fight Club" as the last film they've seen?

In [62]:
fight_club = final_merge2[final_merge2['last_movie'] == 'Fight Club']
ids = fight_club['id'].tolist()
print(f"Users who have 'Fight Club' as last film are: {len(fight_club)} (id: {ids}) ")

Users who have 'Fight Club' as last film are: 3 (id: [2, 18, 20]) 
